In [183]:
import bz2
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, r2_score, make_scorer


In [184]:
with bz2.open('shape_predictor_68_face_landmarks.dat.bz2', 'rb') as compressed_file:
    with open('shape_predictor_68_face_landmarks.dat', 'wb') as decompressed_file:
        decompressed_file.write(compressed_file.read())

# Now you can load the decompressed file
# Load the file using your preferred method, for example, using dlib
import dlib

predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [187]:
# Get the layers (parts) of the shape predictor model
layers = predictor.parts()

# Print the number of layers
print("Number of Layers:", len(layers))

AttributeError: '_dlib_pybind11.shape_predictor' object has no attribute 'parts'

In [7]:

# Load the input image
image = cv2.imread('ivy.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect facial landmarks
landmarks = predictor(gray, dlib.rectangle(0, 0, gray.shape[1], gray.shape[0]))

In [29]:
a = np.array([(point.x, point.y) for point in landmarks.parts()])

In [9]:
for point in landmarks.parts():
    x, y = point.x, point.y
    print(x)
    print(y)

110
137
99
185
101
232
115
286
135
335
161
378
184
425
207
470
238
487
272
482
308
446
344
402
383
359
417
307
440
253
451
197
450
141
123
111
147
115
170
137
189
157
208
175
277
178
305
167
330
155
358
144
388
142
245
233
241
276
238
320
234
364
205
371
220
384
237
393
256
389
273
381
139
186
159
183
184
190
204
214
180
218
154
212
290
224
313
205
339
205
362
213
343
232
316
234
196
409
216
428
232
433
239
440
250
436
270
435
295
420
268
434
248
443
236
444
228
439
211
424
204
418
231
449
238
454
250
452
283
426
247
413
237
415
230
410


In [145]:
with open('updated_data_encoded.json', 'r') as f:
    json_data = f.read()

updated_data = pd.read_json(json_data)

In [133]:
image_path = '/home/jupyter/ML Final/Images/'

def get_face_landmarks(image_name):
    img = cv2.imread(image_path + image_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    landmarks = predictor(gray, dlib.rectangle(0, 0, gray.shape[1], gray.shape[0]))

    return landmarks

In [130]:
cv2.cvtColor(cv2.imread('/home/jupyter/ML Final/Images/img_0.bmp'), cv2.COLOR_BGR2GRAY)

array([[191, 191, 191, ..., 190, 166, 163],
       [191, 191, 191, ..., 185, 172, 160],
       [190, 192, 188, ..., 207, 194, 188],
       ...,
       [101, 106, 109, ..., 101, 105, 106],
       [105, 109, 108, ..., 100, 105, 107],
       [107, 109, 107, ..., 101, 104, 106]], dtype=uint8)

In [134]:
landmark_data = []

for images in updated_data.name:
    
    landmarks = get_face_landmarks(images)
    aray = np.array([(point.x, point.y) for point in landmarks.parts()])
    landmark_data.append(aray)
    


In [146]:
updated_data['landmark_data'] = landmark_data
updated_data.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
4  25.845588  Female            1  img_6.bmp   

                               face_encodings_vggfc6  \
0  [[-1.4681632519, -0.5275202990000001, -22.0670...   
1  [[-0.2039689422, -0.7145279050000001, -62.6996...   
2  [[-0.8742981553, -0.51126647, -30.3239555359, ...   
3  [[-0.713303566, -0.47140464190000003, -33.4862...   
4  [[0.9988322258000001, -0.32367768880000003, -2...   

                            face_encodings_vggresnet  \
0  [[0.0732608438, 0.1081715822, 0.7255821824, 0....   
1  [[0.2826540768, 1.3589353561, 0.0, 0.670866668...   
2  [[0.012803124300000001, 6.6733517647, 0.0, 0.1...   
3  [[0.1181243807, 2.2722120285000003, 4.28302335...   
4  [[0.7783441544, 0.0, 0.25624114270000004, 0.03...   

                                       landmark_data  
0  [[25, 73], [25, 87], [26, 102], [28, 117], [35...  
1  [[28, 75], [27, 91], [27, 107], [31, 121], [39...  
2  [[33, 112], [31, 135], [33, 158], [42, 183], [...  
3  [[19, 47], [17, 59], [18, 71], [20, 83], [24, ...  
4  [[8, 50], [6, 60], [6, 71], [7, 83], [11, 94],...

In [147]:

updated_data['landmark_data'] = updated_data['landmark_data'].apply(lambda x: [item for sublist in x for item in sublist])
updated_data['face_encodings_vggfc6'] = updated_data['face_encodings_vggfc6'].apply(lambda x: [item for sublist in x for item in sublist])
updated_data['face_encodings_vggresnet'] = updated_data['face_encodings_vggresnet'].apply(lambda x: [item for sublist in x for item in sublist])


In [148]:
updated_data.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
4  25.845588  Female            1  img_6.bmp   

                               face_encodings_vggfc6  \
0  [-1.4681632519, -0.5275202990000001, -22.06702...   
1  [-0.2039689422, -0.7145279050000001, -62.69966...   
2  [-0.8742981553, -0.51126647, -30.3239555359, -...   
3  [-0.713303566, -0.47140464190000003, -33.48624...   
4  [0.9988322258000001, -0.32367768880000003, -21...   

                            face_encodings_vggresnet  \
0  [0.0732608438, 0.1081715822, 0.7255821824, 0.0...   
1  [0.2826540768, 1.3589353561, 0.0, 0.6708666682...   
2  [0.012803124300000001, 6.6733517647, 0.0, 0.17...   
3  [0.1181243807, 2.2722120285000003, 4.283023357...   
4  [0.7783441544, 0.0, 0.25624114270000004, 0.034...   

                                       landmark_data  
0  [25, 73, 25, 87, 26, 102, 28, 117, 35, 130, 45...  
1  [28, 75, 27, 91, 27, 107, 31, 121, 39, 135, 48...  
2  [33, 112, 31, 135, 33, 158, 42, 183, 54, 204, ...  
3  [19, 47, 17, 59, 18, 71, 20, 83, 24, 94, 27, 1...  
4  [8, 50, 6, 60, 6, 71, 7, 83, 11, 94, 17, 102, ...

In [121]:
X

face_encodings_vggfc6  \
0     [-1.4681632519, -0.5275202990000001, -22.06702...   
1     [-0.2039689422, -0.7145279050000001, -62.69966...   
2     [-0.8742981553, -0.51126647, -30.3239555359, -...   
3     [-0.713303566, -0.47140464190000003, -33.48624...   
4     [0.9988322258000001, -0.32367768880000003, -21...   
...                                                 ...   
3957  [-1.7679167986, -0.6013143063, -29.7432613373,...   
3958  [1.2071709633, -0.6684975624, -6.7619037628, -...   
3959  [1.4170178175, -0.5360130668, 9.0926294327, -0...   
3960  [-2.3876991272, -0.5760675073, -32.4914283752,...   
3961  [-3.1790702343, -0.468557775, -24.9548416138, ...   

                               face_encodings_vggresnet  \
0     [0.0732608438, 0.1081715822, 0.7255821824, 0.0...   
1     [0.2826540768, 1.3589353561, 0.0, 0.6708666682...   
2     [0.012803124300000001, 6.6733517647, 0.0, 0.17...   
3     [0.1181243807, 2.2722120285000003, 4.283023357...   
4     [0.7783441544, 0.0, 0.25624114270000004, 0.034...   
...                                                 ...   
3957  [1.0812875032, 0.36523407700000005, 3.78530097...   
3958  [0.0, 0.034111402900000004, 0.0448055640000000...   
3959  [0.0, 0.12372529510000001, 0.0404655114, 0.190...   
3960  [0.0784414038, 0.0399528071, 0.834187746, 0.01...   
3961  [0.0103714494, 0.9835363030000001, 0.0, 1.1836...   

                                          landmark_data  
0     [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
1     [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
2     [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
3     [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
4     [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
...                                                 ...  
3957  [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
3958  [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
3959  [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
3960  [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  
3961  [110, 137, 99, 185, 101, 232, 115, 286, 135, 3...  

[3962 rows x 3 columns]

In [136]:
df_train = updated_data[updated_data['is_training'] == 1].reset_index(drop = True)
df_test = updated_data[updated_data['is_training'] == 0].reset_index(drop = True)

In [137]:

X_train = np.array(df_train['landmark_data'].tolist())
X_train = X_train.reshape(X_train.shape[0], -1)

X_test = np.array(df_test['landmark_data'].tolist())
X_test = X_test.reshape(X_test.shape[0], -1)

y_train = np.array(df_train['bmi'])
y_test = np.array(df_test['bmi'])

In [138]:
X_test.shape

(752, 136)

In [139]:
%%time

pearson_corr_scorer = make_scorer(lambda y_true, y_pred: -pearsonr(y_true, y_pred)[0])

param_grid = {'kernel': ['rbf', 'poly'],
    'epsilon': [0.1, 0.5, 1, 5, 10, 15, 20], 
     'C': [0.1, 1, 10, 30, 50, 100]}

# Create the SVR model
model = SVR(kernel = 'rbf')

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=15)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'C': 100, 'epsilon': 5, 'kernel': 'rbf'}
Best score: 0.01183291790256753
CPU times: user 2.91 s, sys: 919 ms, total: 3.83 s
Wall time: 3min 22s


In [149]:
svr_model_landmark = SVR(C=100, kernel='rbf', epsilon=5)
svr_model_landmark.fit(X_train, y_train)
y_pred_landmark = svr_model_landmark.predict(X_test)

mse = mean_squared_error(y_test, y_pred_landmark)
print('Mean squared error: ', mse)

corr, p_value = pearsonr(y_test, y_pred_landmark)
print("Pearson correlation coefficient:", corr)
print("p-value:", p_value)

Mean squared error:  86.29477435686483
Pearson correlation coefficient: 0.18787465196494932
p-value: 2.1060433304483503e-07


In [168]:

# Load the input image
image = cv2.imread('ivy.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect facial landmarks
landmarks = predictor(gray, dlib.rectangle(0, 0, gray.shape[1], gray.shape[0]))

aray = np.array([(point.x, point.y) for point in landmarks.parts()])

In [174]:
aray = [item for sublist in aray for item in sublist]

TypeError: 'numpy.int64' object is not iterable

In [179]:
svr_model_landmark.predict(reshaped_data)


ValueError: X has 2 features, but SVR is expecting 136 features as input.

In [181]:
X_test

array([[ 40, 213,  38, ..., 372, 189, 370],
       [ 47, 139,  43, ..., 271, 120, 269],
       [ 56, 119,  56, ..., 237, 134, 235],
       ...,
       [ 86, 252,  82, ..., 410, 188, 405],
       [ 56, 212,  52, ..., 469, 172, 470],
       [ 26, 235,  41, ..., 480, 241, 482]])